### Práctica de Acceso a bases de datos

# Considerar la gestión de información sobre coches. Para ello se utilizará una base de datos con el siguiente esquema relacional(en negrita aparecen las claves primarias):

marcas (__cifm__, nombre, ciudad)

coches (__codcoche__, nombre, modelo)

marco (__cifm__, __codcoche__)

concesionarios (__cifc__, nombre, ciudad)

distribucion (__cifc__, __codcoche__, cantidad)

clientes (__dni__, nombre, apellido, ciudad)

ventas (__cifc__, __dni__, __codcoche__, color)


Tabla Marcas:

cifm | Nombre | Ciudad
-- | -- |--
0001| seat|Madrid
0002| renault| Barcelona
0003| citroen| Valencia
0004| audi| Madrid
0005| opel| Bilbao
0006| bmw| Barcelona

Tabla Coches:

codcoche | nombre|modelo
-- | --|--
0001| ibiza|glx
0002| ibiza| gti
0003| ibiza| gtd
0004| toledo| gtd
0005| cordoba| gti
0006| megane| 1.6
0007| megane|gti
0008| laguna| gtd
0009| laguna| td
0010| zx| 16v
0011| zx| td
0012| xantia| gtd
0013| a4 |1.8
0014| a4| 2.8
0015| astra| caravan
0016| astra| gti
0017| corsa| 1.4

Tabla Marco:

cifm | codcoche
-- | --
0001| 0001
0001| 0002
0001| 0003
0001| 0004
0001| 0005
0002| 0006
0002| 0007
0002| 0008
0002| 0009
0003| 0010
0003| 0011
0003| 0012
0004| 0013
0004| 0014
0005| 0015
0005| 0016
0005| 0017

Tabla Concesionarios:

cifc | Nombre | Ciudad
-- | -- |--
0001| acar|Madrid
0002| bcar| Madrid
0003| ccar| Barcelona
0004| dcar| Valencia
0005| ecar| Bilbao

Tabla Distribución

cifc | codcoche| cantidad
-- | --| --
0001| 0001 | 3
0001| 0002 | 7
0001| 0003 | 7
0002| 0006 | 5
0002| 0007 | 10
0002| 0008 | 10
0003| 0010 | 5
0003| 0011 | 3
0003| 0012 | 5
0004| 0013 | 10
0004| 0014 | 5


Tabla Clientes:

dni | nombre | apellidos | ciudad
-- | --|--|--
0001| Luis| Garcia | Madrid
0002| Antonio| López | Valencia
0003| Juan| Martín | Madrid
0004| María| Garcia | Madrid
0005| Javier| González | Barcelona
0006| Ana| López | Barcelona
0007| Ana| López | Madrid
0008| Ana| López | Barcelona


Tabla Ventas:

cifc | dni | codcoche | color
-- | --|--|--
0001| 0001 | 0001 | blanco
0001| 0002 | 0005 | rojo
0002| 0003 | 0008 | blanco
0002| 0001 | 0006 | rojo
0003| 0004 | 0011 | rojo
0004| 0005 | 0014 | verde
0004| 0005 | 0013 | azul
0004| 0004 | 0014 | verde

Esta base de datos contiene información sobre marcas y modelos de coches, los concesionarios que venden los coches, los coches distribuidos a los concesionarios y las ventas realizadas por estos últimos a los clientes.

Se pide:

1.Crear un programa en Python que genere utilizando SQLite una base de datos denominada “Vehículos”  que contenga las tablas mostradas más arriba: Marcas, Coche, Marco, Concesionarios, Distribución, Clientes y Ventas con la información que aparece en cada una de ellas. [2,5 puntos].

2.Crear un programa en Python que resuelva las siguientes consultas SQL [7,5 puntos]. Se debe mostrar por pantalla el resultado de cada consulta:
 
   * Obtener el nombre y el apellido de los clientes que han comprado como mínimo un coche de color ‘verde’ o un coche de color ‘azul’,

   * Obtener el nombre de los concesionarios que sólo han vendido coches al cliente con dni igual a 0001

   * Obtener los codcoche y la media de los valores del atributo cantidad para cada codcoche que aparece en la relación distribucion para los casos en que la media sea menor que 15

   * Obtener para cada valor de dni de los clientes el número de veces que aparece en la relación ventas.

   * Obtener el dni de los clientes de nombre ‘pepe’ o de nombre ‘luis’ (no se admiten valores repetidos)

   * Obtener el cifc de todos los concesionarios cuyo atributo cantidad en la relación distribucion no está comprendido entre 10 y 18 ambos inclusive (es decir, los valores menores que 10 y mayores que 18)

#### Normas de entrega

* Fecha tope de entrega: 17/10/2018
* La entrega se realizará subiendo al campus virtual un notebook de Jupyter con la solución. El archivo tendrá como nombre BasesDatos_GrupoX donde X será el número de grupo correspondiente.


In [6]:
def crear_conectar_DB(conexion):
    cur=conexion.cursor()
    print('\033[1m',"CREANDO LA BASE DE DATOS...",'\033[0m', "\n")
    cur.execute("DROP TABLE IF EXISTS Marcas")
    cur.execute("CREATE TABLE Marcas(cifm INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL, nombre TEXT, ciudad TEXT)")

    cur.execute("DROP TABLE IF EXISTS Coches")
    cur.execute("create table Coches(codigo INTEGER PRIMARY KEY AUTOINCREMENT, nombre TEXT, modelo TEXT)")

    cur.execute("DROP TABLE IF EXISTS Marco")
    cur.execute("create TABLE Marco(cifm NUMBER, codcoche NUMBER, FOREIGN KEY(cifm) REFERENCES Marcas(cifm), FOREIGN KEY(codcoche) REFERENCES Coches(codigo),  PRIMARY KEY(cifm, codcoche))")

    cur.execute("DROP TABLE IF EXISTS Concesionarios")
    cur.execute("create table Concesionarios(cifc INTEGER PRIMARY KEY AUTOINCREMENT, nombre TEXT, ciudad TEXT)")

    cur.execute("DROP TABLE IF EXISTS Distribucion")
    cur.execute("create table Distribucion(cifc  NUMBER, codcoche NUMBER, cantidad NUMBER, FOREIGN KEY(cifc) REFERENCES Concesionarios(cifc), FOREIGN KEY(codcoche) REFERENCES Coches(codigo), PRIMARY KEY(cifc, codcoche))")

    cur.execute("DROP TABLE IF EXISTS Clientes")
    cur.execute("create table Clientes(dni TEXT PRIMARY KEY, nombre TEXT, apellidos TEXT, ciudad TEXT)")

    cur.execute("DROP TABLE IF EXISTS Ventas")
    cur.execute("CREATE TABLE Ventas(cifc  NUMBER, dni TEXT, codcoche NUMBER, color TEXT,  FOREIGN KEY(cifc) REFERENCES Concesionarios(cifm),  FOREIGN KEY(dni) REFERENCES Clientes(dni),  FOREIGN KEY(codcoche) REFERENCES Coches(codigo),  PRIMARY KEY(cifc, dni, codcoche))")
    cur.close()
    conexion.commit()


def insert(nombreTabla, atributos, datosTabla, conexion):
    cur = conexion.cursor()
    query="INSERT INTO "+nombreTabla + " ("
    parametros = ""
    for atributo in atributos:
        query += atributo + ","
        parametros += "?, "
    query = query.rstrip(",")
    
    parametros = parametros.rstrip(", ")
    query += ") VALUES (" + parametros + ")"
    cur.executemany(query,datosTabla)
    print ("Insertadas", cur.rowcount, "filas en tabla ", nombreTabla)
    cur.close()
    
def insertarDatos(conn):
    camposMarcas = ["nombre", "ciudad"]
    camposCoches = ["nombre", "modelo"]
    camposMarco = ["cifm","codcoche"]
    camposConcesionarios = ["nombre","ciudad"]
    camposDistribucion = ["cifc","codcoche","cantidad"]
    camposClientes = ["dni","nombre","apellidos","ciudad"]
    camposVentas = ["cifc","dni","codcoche","color"]


    marcas = [('seat','Madrid'),('renault', 'Barcelona'),('citroen', 'Valencia'),('audi', 'Madrid'),('opel', 'Bilbao'),('bmw', 'Barcelona')]
    coches = [('ibiza','glx'),('ibiza', 'gti'),('ibiza', 'gtd'),('toledo', 'gtd'),('cordoba', 'gti'),('megane', '1.6'),('megane', 'gti'),('laguna', 'gtd'),('laguna', 'td'),('zx', '16v'),('zx', 'td'),('xantia', 'gtd'),('a4', '1.8'),('a4', '2.8'),('astra', 'caravan'),('astra', 'gti'),('corsa', '1.4')]
    marco = [('1','1'),('1','2'),('1','3'),('1','4'),('1','5'),('2','6'),('2','7'),('2','8'),('2','9'),('3','10'),('3','11'),('3','12'),('4','13'),('4','14'),('5','15'),('5','16'),('5','17')]
    concesionarios = [('acar', 'Madrid'),('bcar', 'Madrid'),('ccar', 'Barcelona'),('dcar', 'Valencia'),('ecar', 'Bilbao')]
    distribucion = [('1', '1', '3'),('1', '2', '7'),('1', '3', '7'),('2', '6', '5'),('2', '7', '10'),('2', '8', '10'),('3', '10', '5'),('3', '11', '3'),('3', '12', '5'),('4', '13', '10'),('4', '14', '5')]
    clientes = [('1', 'Luis', 'Garcia', 'Madrid'),('2', 'Antonio', 'Lopez', 'Valencia'),('3', 'Juan', 'Martin', 'Madrid'),('4', 'Maria', 'Garcia', 'Madrid'),('5', 'Javier', 'Gonzalez', 'Barcelona'),('6', 'Ana', 'Lopez', 'Barcelona'),('7', 'Ana', 'Lopez', 'Madrid'),('8', 'Ana', 'Garcia', 'Barcelona')]
    ventas = [('1', '1', '1', 'blanco'),('1', '2', '5', 'rojo'),('2', '3', '8', 'blanco'),('2', '1', '6', 'rojo'),('3', '4', '11', 'rojo'),('4', '5', '14', 'verde'),('4', '5', '13', 'azul'),('4', '4', '14', 'verde')]

    insert("MARCAS", camposMarcas, marcas, conn)
    insert("COCHES", camposCoches, coches, conn)
    insert("MARCO", camposMarco, marco, conn)
    insert("CONCESIONARIOS", camposConcesionarios, concesionarios, conn)
    insert("DISTRIBUCION", camposDistribucion, distribucion, conn)
    insert("CLIENTES", camposClientes, clientes, conn)
    insert("VENTAS", camposVentas, ventas, conn)
    conn.commit()


def consultas(conexion):
    
    cur=conexion.cursor()
    #CONSULTAS--------------------------
    
    print('\033[1m',"CONSULTAS:",'\033[0m', "\n")

    #1 - Obtener el nombre y el apellido de los clientes que han comprado como mínimo un coche de color ‘verde’ o un 
    #coche de color ‘azul’,
    cur.execute("SELECT DISTINCT nombre, apellidos FROM Clientes WHERE dni IN (SELECT dni FROM ventas WHERE (color = 'verde' OR color = 'azul'))")
    print('\033[1m',"PRIMERA CONSULTA: Clientes que han comprado un coche verde o rojo",'\033[0m', "\n")
    resultados = cur.fetchall()
    if not resultados:
        print("No hay resultados")
    else:
        for (nombre, apellidos) in resultados:
            print ("Nombre:", nombre, "Apellidos:", apellidos)
            print ("\n")


    #2 - Obtener el nombre de los concesionarios que sólo han vendido coches al cliente con dni igual a 0001
    cur.execute("SELECT nombre FROM Concesionarios WHERE cifc IN (SELECT DISTINCT cifc FROM Ventas WHERE cifc NOT IN (SELECT DISTINCT cifc FROM Ventas WHERE dni != 1))")
    print ('\033[1m', "SEGUNDA CONSULTA: Nombre de los concesionarios que solo han vendido al cliente con dni = 1",'\033[0m', "\n")
    resultados = cur.fetchall()
    if not resultados:
        print("No hay resultados\n")
    else:
        for nombre in cur.fetchall():
                print ("Concesionario:", nombre)
                print ("\n")



    #3 - Obtener los codcoche y la media de los valores del atributo cantidad para cada codcoche que aparece en la 
    #relación distribucion para los casos en que la media sea menor que 15
    cur.execute("SELECT codcoche, AVG(cantidad) FROM Distribucion WHERE codcoche NOT IN (SELECT codcoche FROM Distribucion GROUP BY codcoche HAVING avg(cantidad)>=15) GROUP BY codcoche")
    
    ############ Creo que de esta forma es más eficiente porque solo hace la media y agrupa una vez pero echadle un ojo
    cur.execute("SELECT * FROM (SELECT codcoche, avg(cantidad) as media FROM Distribucion GROUP BY codcoche) WHERE media < 15")

    print ('\033[1m', "TERCERA CONSULTA: Código del coche y la media de los valores del atributo cantidad con media < 15", '\033[0m',"\n")
    resultados = cur.fetchall()
    if not resultados:
        print("No hay resultados\n")
    else:
        for (codcoche, cantidad) in resultados:
            print ("Codigo Coche:", codcoche, "Cantidad Media:", cantidad)
            print ("\n")


    #4 - Obtener para cada valor de dni de los clientes el número de veces que aparece en la relación ventas.
    cur.execute("SELECT dni, COUNT(*) FROM ventas GROUP BY dni")
    print ('\033[1m', "CUARTA CONSULTA: Número de veces que cada cliente ha comprado", '\033[0m',"\n")
    resultados = cur.fetchall()
    if not resultados:
        print("No hay resultados\n")
    else:
        for (dni, compras) in resultados:
            print ("DNI:", dni, "Número de compras:", compras)
            print ("\n")


    #5 - Obtener el dni de los clientes de nombre ‘pepe’ o de nombre ‘luis’ (no se admiten valores repetidos)
    cur.execute("SELECT DISTINCT dni FROM clientes WHERE (nombre = 'Pepe' OR nombre = 'Luis')")
    print ('\033[1m', "QUINTA CONSULTA: DNI de los clientes de nombre Pepe o de nombre Luis",'\033[0m', "\n")
    resultados = cur.fetchall()
    if not resultados:
        print("No hay resultados\n")
    else:
        for dni in resultados:
            print ("DNI:", dni)
            print ("\n")

    #6 - Obtener el cifc de todos los concesionarios cuyo atributo cantidad en la relación distribucion no está comprendido 
    #entre 10 y 18 ambos inclusive (es decir, los valores menores que 10 y mayores que 18)
    cur.execute("SELECT DISTINCT cifc FROM Distribucion WHERE cifC NOT IN (SELECT cifc FROM Distribucion GROUP BY cifc HAVING SUM(cantidad) < 10 OR SUM(cantidad) > 18)")
    
    ############## Creo que la SEXTA esta sacando los valores al reves en la consulta, deberia ser IN en vez de
    # NOT IN porque en la subconsulta esta sacando los cifc que nos interesan, y creo que esta que he hecho yo
    # esta algo mas compacto pero como querais 
    cur.execute("SELECT cifc FROM (SELECT cifc, sum(cantidad) as suma FROM Distribucion GROUP BY cifc) WHERE suma <= 10 or suma >= 18") 

    print ('\033[1m', "SEXTA CONSULTA: ", '\033[0m',"\n")
    resultados = cur.fetchall()
    if not resultados:
        print("No hay resultados\n")
    else:
        for cifc in resultados:
            print ("CIFC:", cifc)
            print ("\n")
    cur.close()

def salir(conexion):
    conexion.commit()
    conexion.close()

In [7]:
import sqlite3
conn=sqlite3.connect("vehiculos.sqlite3")
crear_conectar_DB(conn)
insertarDatos(conn)
consultas(conn)
salir(conn)

 CREANDO LA BASE DE DATOS...  

Insertadas 6 filas en tabla  MARCAS
Insertadas 17 filas en tabla  COCHES
Insertadas 17 filas en tabla  MARCO
Insertadas 5 filas en tabla  CONCESIONARIOS
Insertadas 11 filas en tabla  DISTRIBUCION
Insertadas 8 filas en tabla  CLIENTES
Insertadas 8 filas en tabla  VENTAS
 CONSULTAS:  

 PRIMERA CONSULTA: Clientes que han comprado un coche verde o rojo  

Nombre: Maria Apellidos: Garcia


Nombre: Javier Apellidos: Gonzalez


 SEGUNDA CONSULTA: Nombre de los concesionarios que solo han vendido al cliente con dni = 1  

No hay resultados

 TERCERA CONSULTA: Código del coche y la media de los valores del atributo cantidad con media < 15  

Codigo Coche: 1 Cantidad Media: 3.0


Codigo Coche: 2 Cantidad Media: 7.0


Codigo Coche: 3 Cantidad Media: 7.0


Codigo Coche: 6 Cantidad Media: 5.0


Codigo Coche: 7 Cantidad Media: 10.0


Codigo Coche: 8 Cantidad Media: 10.0


Codigo Coche: 10 Cantidad Media: 5.0


Codigo Coche: 11 Cantidad Media: 3.0


Codigo Coche: 12 C

In [ ]:
# Scrip para la BD en sql

drop table if exists Marco;
drop table if exists Distribucion;
drop table if exists Ventas;
drop table if exists Clientes;
drop table if exists Concesionarios;
drop table if exists Coches;
drop table if exists Marcas;

CREATE TABLE Marcas(

	cifm int AUTO_INCREMENT,
	nombre varchar(20),
	ciudad varchar(40),
	PRIMARY KEY (cifm)
);

CREATE TABLE Coches(

	codcoche int AUTO_INCREMENT,
	nombre varchar(20),
	modelo varchar(30),
	PRIMARY KEY (codcoche)
);

CREATE TABLE Concesionarios(

	cifc int,
	nombre varchar(30),
	ciudad varchar(30),
	PRIMARY KEY(cifc)
);

CREATE TABLE Clientes(

	dni int AUTO_INCREMENT,
	nombre varchar(20),
	apellido varchar(40),
	ciudad varchar(40),
	PRIMARY KEY (dni)
);

CREATE TABLE Ventas(

	cifc int,
	dni int,
	codcoche int,
	color varchar(15),
	PRIMARY KEY(cifc, dni, codcoche),
	FOREIGN KEY (cifc) REFERENCES Concesionarios(cifc),
	FOREIGN KEY (dni) REFERENCES Clientes(dni),
	FOREIGN KEY (codcoche) REFERENCES Coches(codcoche)
);

CREATE TABLE Distribucion(

	cifc int,
	codcoche int,
	cantidad int,
	PRIMARY KEY (cifc, codcoche),
	FOREIGN KEY (cifc) REFERENCES Concesionarios(cifc),
	FOREIGN KEY (codcoche) REFERENCES Coches(codcoche)
);

CREATE TABLE Marco(

	cifm int,
	codcoche int,
	PRIMARY KEY (cifm, codcoche),
	FOREIGN KEY (cifm) REFERENCES Marcas(cifm),
	FOREIGN KEY (codcoche) REFERENCES Coches(codcoche)
);


INSERT INTO Marcas
VALUES (1, "seat", "Madrid");
INSERT INTO Marcas
VALUES (2, "renault", "Barcelona");
INSERT INTO Marcas
VALUES (3, "citroen", "Valencia");
INSERT INTO Marcas
VALUES (4, "audi", "Madrid");
INSERT INTO Marcas
VALUES (5, "opel", "Bilbao");
INSERT INTO Marcas
VALUES (6, "bmw", "Barcelona");



INSERT INTO Coches
VALUES (1, "ibiza", "glx");
INSERT INTO Coches
VALUES (2, "ibiza", "gti");
INSERT INTO Coches
VALUES (3, "ibiza", "gtd");
INSERT INTO Coches
VALUES (4, "toledo", "gtd");
INSERT INTO Coches
VALUES (5, "cordoba", "gti");
INSERT INTO Coches
VALUES (6, "megane", "1.6");
INSERT INTO Coches
VALUES (7, "megane", "gti");
INSERT INTO Coches
VALUES (8, "laguna", "gtd");
INSERT INTO Coches
VALUES (9, "laguna", "td");
INSERT INTO Coches
VALUES (10, "zx", "16v");
INSERT INTO Coches
VALUES (11, "zx", "td");
INSERT INTO Coches
VALUES (12, "xantia", "gtd");
INSERT INTO Coches
VALUES (13, "a4", "1.8");
INSERT INTO Coches
VALUES (14, "a4", "2.8");
INSERT INTO Coches
VALUES (15, "astra", "caravan");
INSERT INTO Coches
VALUES (16, "astra", "gti");
INSERT INTO Coches
VALUES (17, "corsa", "1.4");



INSERT INTO Marco
VALUES (1, 1);
INSERT INTO Marco
VALUES (1, 2);
INSERT INTO Marco
VALUES (1, 3);
INSERT INTO Marco
VALUES (1, 4);
INSERT INTO Marco
VALUES (1, 5);
INSERT INTO Marco
VALUES (2, 6);
INSERT INTO Marco
VALUES (2, 7);
INSERT INTO Marco
VALUES (2, 8);
INSERT INTO Marco
VALUES (2, 9);
INSERT INTO Marco
VALUES (3, 10);
INSERT INTO Marco
VALUES (3, 11);
INSERT INTO Marco
VALUES (3, 12);
INSERT INTO Marco
VALUES (4, 13);
INSERT INTO Marco
VALUES (4, 14);
INSERT INTO Marco
VALUES (5, 15);
INSERT INTO Marco
VALUES (5, 16);
INSERT INTO Marco
VALUES (5, 17);


INSERT INTO Concesionarios
VALUES (1, "acar", "madrid");
INSERT INTO Concesionarios
VALUES (2, "bcar", "madrid");
INSERT INTO Concesionarios
VALUES (3, "ccar", "Barcelona");
INSERT INTO Concesionarios
VALUES (4, "dcar", "Valencia");
INSERT INTO Concesionarios
VALUES (5, "ecar", "Bilbao");


INSERT INTO Distribucion
VALUES (1, 1, 3);
INSERT INTO Distribucion
VALUES (1, 2, 7);
INSERT INTO Distribucion
VALUES (1, 3, 7);
INSERT INTO Distribucion
VALUES (2, 6, 5);
INSERT INTO Distribucion
VALUES (2, 7, 10);
INSERT INTO Distribucion
VALUES (2, 8, 10);
INSERT INTO Distribucion
VALUES (3, 10, 5);
INSERT INTO Distribucion
VALUES (3, 11, 3);
INSERT INTO Distribucion
VALUES (3, 12, 5);
INSERT INTO Distribucion
VALUES (4, 13, 10);
INSERT INTO Distribucion
VALUES (4, 14, 5);

INSERT INTO Clientes
VALUES (1, "Luis", "Garcia", "Madrid");
INSERT INTO Clientes
VALUES (2, "Antonio", "Lopez", "Valencia");
INSERT INTO Clientes
VALUES (3, "Juan", "Martin", "Madrid");
INSERT INTO Clientes
VALUES (4, "Maria", "Garcia", "Madrid");
INSERT INTO Clientes
VALUES (5, "Javier", "Gonzalez", "Barcelona");
INSERT INTO Clientes
VALUES (6, "Ana", "Lopez", "Barcelona");
INSERT INTO Clientes
VALUES (7, "Ana", "Lopez", "Madrid");
INSERT INTO Clientes
VALUES (8, "Ana", "Lopez", "Barcelona");


INSERT INTO Ventas
VALUES (1, 1, 1, "blanco");
INSERT INTO Ventas
VALUES (1, 2, 5, "rojo");
INSERT INTO Ventas
VALUES (2, 3, 8, "blanco");
INSERT INTO Ventas
VALUES (2, 1, 6, "rojo");
INSERT INTO Ventas
VALUES (3, 4, 11, "rojo");
INSERT INTO Ventas
VALUES (4, 5, 14, "verde");
INSERT INTO Ventas
VALUES (4, 5, 13, "rojo");
INSERT INTO Ventas
VALUES (4, 4, 14, "verde");